In [ ]:
import pandas as pd
import os

## Setup

- Create a directory `data/medical/data` with subdirectories `data/medical/data/raw` and `data/medical/processed`
- Extract the static data fields using the python script in the cell below. NOTE: this assumes that you have an `enc_ukb` file from UK Biobank and the software programs for working with this type of file. You will need to update `data_command` in the script below to provide the correct filepaths for your `enc_ukb` file and software
- The rest of the code assumes you have the UK Biobank Primary Care Linked Data stored in an SQLite database called `ehr.db` for the tables `gp_clinical` and `gp_scripts`. See [UK Biobank Resource 591](https://biobank.ndph.ox.ac.uk/showcase/refer.cgi?id=591) for further details

In [ ]:
# creates a file with the fields for extraction
df = pd.read_csv('inputs/covariates.txt')
fields = df[df['Type']=='static']['Code'].tolist()
with open('data/fields.txt', 'w') as f:
    for field in fields:
        f.write("%s\n" % field)

data_command = 'ukbconv ukbxxxx.enc_ukb csv -ifields.txt -oukbxxxx' # TODO update this command with your filepaths
os.system(data_command) # alternatively, copy the command into your bash shell

In [ ]:
# load required packages
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import random
import numpy as np
from scipy import stats

In [ ]:
# specify filepaths and database location
# TODO update these for your filepaths

OUTPATH = 'data/raw'
DATAPATH = 'data/ukbxxxxcsv'

df = pd.read_csv(DATAPATH)
fields_df = pd.read_csv('inputs/covariates.txt')
covariate_fields_df = fields_df[fields_df['Variable']=='covariate']
clinical_fields_df = fields_df[fields_df['Variable']=='clinical']
diagnosis_fields_df = fields_df[fields_df['Variable']=='diagnosis']
resp_df = pd.read_csv('inputs/response.txt')
tre_df = pd.read_csv('inputs/treatment.txt')

# connection to db
DBPATH = 'data/ehr/ehr.db'
conn = sqlite3.connect(DBPATH)
conn.text_factory = lambda b: b.decode(errors = 'ignore')

In [ ]:
# utility function for removing outliers from a column in a pandas dataframe
outlier_df = lambda df, col: df[(np.abs(stats.zscore(df[col])) < 3)]

## Preprocess patient list and static covariates

In [ ]:
## format static data (diagnosis covariates and other covariates)
df = pd.read_csv(DATAPATH)
diagnosis_var = '130708-0.0'
data = df[['eid',diagnosis_var]].rename(columns={diagnosis_var:'T2D_diagnosis_date'}).dropna()
field_list = dict(zip(covariate_fields_df['Code'], covariate_fields_df['Description']))

if '31' in field_list:
    # sex
    tmp = df[['eid', '31-0.0']].rename(columns={'31-0.0':field_list['31']})
    data = pd.merge(data, tmp, on='eid')

if '34' in field_list:
    # year of birth
    tmp = df[['eid', '34-0.0']].rename(columns={'34-0.0':field_list['34']})
    data = pd.merge(data, tmp, on='eid')

## patient cohort: get patients with T2D diagnosis
patient_list = df['eid'].tolist()
data.to_csv('{}/patient_covariates.csv'.format(OUTPATH), index=None)
data


## Preprocess response variable

In [ ]:
table='gp_clinical'
read2_codes = ', '.join(["'{}'".format(x) for x in resp_df[resp_df['Type']=='read2']['Code']])
read3_codes = ', '.join(["'{}'".format(x) for x in resp_df[resp_df['Type']=='read3']['Code']])

if len(read2_codes)>0 and len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE (read_2 IN ({}) OR read_3 IN ({}))'.format(table, read2_codes, read3_codes)
elif len(read2_codes)>0:
    command = 'SELECT * FROM {} WHERE read_2 IN ({})'.format(table, read2_codes)
elif len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE read_3 IN ({})'.format(table, read3_codes)

data = pd.read_sql_query(command, conn)

# data cleaning
data['y'] = pd.to_numeric(data['value1'], errors='coerce')
data = data[['eid','event_dt','y']]
data = data[data['eid'].astype('int').isin(patient_list)]
data = data.dropna()
data = data[~(data['event_dt']=='')]
data['event_dt'] = pd.to_datetime(data['event_dt'], format='%d/%m/%Y')
data = outlier_df(data, 'y')

data = data[data['y']>0]
data.to_csv('{}/response.csv'.format(OUTPATH), index=None)

## Preprocess treatment variable

In [ ]:
table='gp_scripts'

read2_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='read_2']['Code']])
bnf_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='bnf_code']['Code']])
dmd_codes = ', '.join(["'{}'".format(x) for x in tre_df[tre_df['Type']=='dmd_code']['Code']])

command = 'SELECT * FROM {} WHERE (read_2 IN ({}) OR bnf_code IN ({}) OR dmd_code IN ({}))'.format(table, read2_codes, bnf_codes, dmd_codes)

data = pd.read_sql_query(command, conn)

# map the codes to the drug label
code_map = dict(zip(tre_df['Code'],tre_df['Label'])) # use read 2 codes because drug name is blank
t1 = data[data['read_2']!='']
t1['treatment'] = t1['read_2'].map(code_map)
t2 = data[data['read_2']=='']
code_map = dict(zip(tre_df['Description'],tre_df['Label'])) # use description because other drugs had same codes
t2['treatment'] = t2['drug_name'].map(code_map)

data = pd.concat([t1,t2])

# try to match remaining names

treatments = {'atorvastatin':'atorvastatin',
              'lipitor':'atorvastatin', 
              'fluvastatin':'fluvastatin', 
              'lescol':'fluvastatin', 
              'lovastatin':'lovastatin',
              'altoprev':'lovastatin', 
              'pitavastatin':'pitavastatin', 
              'livalo':'pitavastatin', 
              'zypitamag':'pitavastatin', 
              'pravastatin':'pravastatin',
              'pravachol':'pravastatin', 
              'rosuvastatin':'rosuvastatin',
              'crestor':'rosuvastatin',
              'ezallor':'rosuvastatin', 
              'simvastatin':'simvastatin', 
              'zocor':'simvastatin'}

t1 = data[~data['treatment'].isna()]
t2 = data[data['treatment'].isna()]

t2['treatment'] = t2['drug_name'].apply(lambda x: next((v for k, v in treatments.items() if k in x.lower()), None))

# remove any drugs that seem to be billed across wrong codes (could also check manually)
t2 = t2[~t2['treatment'].isna()]
data = pd.concat([t1,t2])

# data cleaning
data = data[['eid','issue_date','treatment']]
data = data[data['eid'].astype('int').isin(patient_list)]
data = data.dropna()
data = data[~(data['issue_date']=='')]
data['issue_date'] = pd.to_datetime(data['issue_date'], format='%d/%m/%Y')
data.rename(columns={'issue_date':'event_dt'}, inplace=True)

data.to_csv('{}/treatment.csv'.format(OUTPATH), index=None)

## Preprocess clinical covariates

In [ ]:
table='gp_clinical'
read2_codes = ', '.join(["'{}'".format(x) for x in clinical_fields_df[clinical_fields_df['Type']=='read2']['Code']])
read3_codes = ', '.join(["'{}'".format(x) for x in clinical_fields_df[clinical_fields_df['Type']=='read3']['Code']])

if len(read2_codes)>0 and len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE (read_2 IN ({}) OR read_3 IN ({}))'.format(table, read2_codes, read3_codes)
elif len(read2_codes)>0:
    command = 'SELECT * FROM {} WHERE read_2 IN ({})'.format(table, read2_codes)
elif len(read3_codes)>0:
    command = 'SELECT * FROM {} WHERE read_3 IN ({})'.format(table, read3_codes)
    
data = pd.read_sql_query(command, conn)

# data cleaning
code_map = dict(zip(clinical_fields_df['Code'],clinical_fields_df['Description']))
data['read_3'] = data['read_3'].map(code_map)
data['read_2'] = data['read_2'].map(code_map)
data['read_3'] = data['read_3'].fillna(data['read_2'])
data['value'] = pd.to_numeric(data['value1'], errors='coerce')
data = data[['eid','event_dt','read_3','value']]
data = data[data['eid'].astype('int').isin(patient_list)]
data = data.dropna()
data = data[~(data['event_dt']=='')]
data['event_dt'] = pd.to_datetime(data['event_dt'], format='%d/%m/%Y')
data.rename(columns={'read_3':'variable'}, inplace=True)

# data should not be zero-valued
data = data[data['value']>0] 

# remove outliers for each variable
final_data = pd.DataFrame()

for variable in data['variable'].unique():
    final_data = pd.concat([final_data,outlier_df(data[data['variable']==variable], 'value')])

final_data.to_csv('{}/covariate_clinical.csv'.format(OUTPATH), index=None)


## Preprocess diagnosis covariates

In [ ]:
df = pd.read_csv(DATAPATH)
field_list = dict(zip(diagnosis_fields_df['Code'], diagnosis_fields_df['Description']))

data = df[['eid']+['{}-0.0'.format(k) for k in field_list.keys()]]
data.index = data['eid']
data.drop(columns='eid', inplace=True)
data = data.unstack().reset_index().dropna().rename(columns={'level_0':'variable',0:'event_dt'})
data['variable'] = data['variable'].str.split('-').str[0]
data['variable'] = data['variable'].map(field_list)
data = data[['eid','variable','event_dt']]
data.to_csv('{}/covariate_diagnosis.csv'.format(OUTPATH), index=None)

## Assemble the dataset

### Add patients, labels and age, sex covariates

In [ ]:
df = pd.read_csv('{}/response.csv'.format(OUTPATH))
df['event_dt'] = pd.to_datetime(df['event_dt'], format='%Y/%m/%d')

# merge with patient covariates and add age column
df_core = pd.read_csv('{}/patient_covariates.csv'.format(OUTPATH))
df = pd.merge(df, df_core, on='eid', how='left')

df['Age'] = df['event_dt'].dt.year - df['Year of birth']
df.drop(columns='Year of birth', inplace=True)
df = df[(df['Age']>18)&(df['Age']<100)]

# exclude patients with number of samples less than MIN_SAMPLES
MIN_SAMPLES = 10
tmp = df.groupby('eid').count()
patient_list = tmp[tmp['y']>=MIN_SAMPLES].index.tolist()
df = df[df['eid'].isin(patient_list)]

### Add treatment variables

In [ ]:
# add treatments reported up to T_DAYS days before event_dt
T_DAYS = 60
df_tre = pd.read_csv('{}/treatment.csv'.format(OUTPATH))
df_tre = df_tre[df_tre['eid'].isin(patient_list)]
df_tre['event_dt'] = pd.to_datetime(df_tre['event_dt'], format='%Y/%m/%d')
df_tre['treatment'] = 'Treatment_' + df_tre['treatment']

tmp = pd.merge(df[['eid','event_dt']], df_tre, on='eid', how='left').dropna()
tmp['t'] = (tmp['event_dt_x'] - tmp['event_dt_y']).dt.days # time between event and treatment
tmp = tmp[(tmp['t']>0)&(tmp['t']<=T_DAYS)]

tmp = tmp.sort_values(by='t').drop_duplicates(subset=['eid','event_dt_x','treatment'], keep='last')
tmp = tmp.drop(columns=['event_dt_y']).rename(columns={'event_dt_x':'event_dt'})
tmp = pd.concat([tmp.drop(columns=['treatment','t']), pd.get_dummies(tmp['treatment'])], axis=1).groupby(['eid','event_dt']).max().reset_index()

df = pd.merge(df, tmp, on=['eid','event_dt'], how='left').fillna(0)

### Add diagnosis variables

In [ ]:
# add diagnosis covariates if reported for the patient prior to event_dt
df_diag = pd.read_csv('{}/covariate_diagnosis.csv'.format(OUTPATH))
df_diag= df_diag[df_diag['eid'].isin(patient_list)]
df_diag['event_dt'] = pd.to_datetime(df_diag['event_dt'], format='%Y/%m/%d')
df_diag['variable'] = 'Diagnosis_' + df_diag['variable']
df_diag = df_diag.sort_values(by='event_dt', ascending=False).drop_duplicates(subset=['eid','variable'], keep='first')
tmp = pd.merge(df[['eid','event_dt']], df_diag, on='eid', how='left').dropna()
tmp['first_diagnosis'] = (tmp['event_dt_x'] - tmp['event_dt_y']).dt.days
tmp = tmp[tmp['first_diagnosis']>0]
tmp['value'] = 1
tmp = tmp.pivot_table(index=['eid','event_dt_x'],columns='variable',values='value',aggfunc=np.mean).reset_index().rename(columns={'event_dt_x':'event_dt'})
df = pd.merge(df, tmp, on=['eid','event_dt'], how='left').fillna(0)

### Add lab variables

In [ ]:
# clinical covariates are based on most recent value in data in past T_DAYS days
# if no values recorded then remove row from dataset
T_DAYS = 10*12*30 # past 10 years
df_clin = pd.read_csv('{}/covariate_clinical.csv'.format(OUTPATH))
df_clin= df_clin[df_clin['eid'].isin(patient_list)]
df_clin['event_dt'] = pd.to_datetime(df_clin['event_dt'], format='%Y/%m/%d')
df_clin['variable'] = 'Clinical_' + df_clin['variable']

tmp = pd.merge(df[['eid','event_dt']], df_clin, on='eid', how='left').dropna()
tmp['time'] = (tmp['event_dt_x'] - tmp['event_dt_y']).dt.days
tmp = tmp[(tmp['time']>0)&(tmp['time']<=T_DAYS)].sort_values(by='time').groupby(['eid','event_dt_x','variable']).first().reset_index()
tmp = tmp.pivot_table(index=['eid','event_dt_x'],columns='variable',values='value',aggfunc=np.mean).reset_index().rename(columns={'event_dt_x':'event_dt'})
df = pd.merge(df, tmp, on=['eid','event_dt'], how='left')

# remove patients with less than MIN_SAMPLES samples
df = df.dropna()
tmp = df.groupby('eid').count()
patient_list = tmp[tmp['event_dt']>=MIN_SAMPLES].index.tolist()
df = df[df['eid'].isin(patient_list)]

### Final cleanup and formatting

In [ ]:
cutoff_date = '2010-01-01'
# only consider patients diagnosed before cutoff date, and their data after cutoff date
df = df[(df['T2D_diagnosis_date']<cutoff_date)&(df['event_dt']>=cutoff_date)]
tmp = df.groupby('eid').count().reset_index()
patient_list = tmp[tmp['event_dt']>=MIN_SAMPLES]['eid'].tolist()
df = df[df['eid'].isin(patient_list)]

In [ ]:
df = df.drop(columns='Diagnosis_E11 T2D')

In [ ]:
# fix issue where some Creatinine values are given in wrong unit
df['Clinical_Creatinine'] = df['Clinical_Creatinine'].apply(lambda x : x/1000 if x>1000 else x)

In [ ]:
# remove patients with fluvastatin - too few samples
eid_remove = df[df['Treatment_fluvastatin']==1]['eid'].unique().tolist()
df = df[~df['eid'].isin(eid_remove)]
df = df.drop(columns='Treatment_fluvastatin')

In [ ]:
# save clean data
df.to_csv('data/processed/ukbb.csv',index=None)

## Create random splits for experiments

In [ ]:
data = pd.read_csv('data/processed/ukbb.csv')

In [ ]:
# create train/val/test datasets for experiments

NUMTASKS_TRAIN = 205
NUMTASKS_VAL = 30
NUMTASKS_TEST = 30
NUMSAMPLES_SPT = 5
NUMSAMPLES_QRY = 5

PATIENTS = data['eid'].unique().tolist()

for dataset in range(1,4):

    random.shuffle(PATIENTS)
    train_tasks = PATIENTS[0:NUMTASKS_TRAIN]
    val_tasks = PATIENTS[NUMTASKS_TRAIN:NUMTASKS_TRAIN+NUMTASKS_VAL]
    test_tasks = PATIENTS[NUMTASKS_TRAIN+NUMTASKS_VAL:]

    meta_train_type = {'train':0, 'val':1, 'test':2}
    patient_list = {'train':train_tasks, 'val':val_tasks, 'test':test_tasks}
    
    patient_map = dict(zip(PATIENTS,range(len(PATIENTS))))
    
    df = data.sample(frac=1)
    new_df = pd.DataFrame()
    for datatype in ['train', 'val', 'test']:
        add_data = df[df['eid'].isin(patient_list[datatype])]
        add_data['meta_train'] = meta_train_type[datatype]
        add_data['keep'] = (add_data.groupby('eid').cumcount() < NUMSAMPLES_SPT+NUMSAMPLES_QRY).astype(int)
        add_data = add_data[add_data['keep']==1]
        add_data = add_data.drop(columns='keep')
        add_data['task_train'] = (add_data.groupby('eid').cumcount() < NUMSAMPLES_SPT).astype(int)
        add_data['task'] = add_data['eid'].map(patient_map)
        rename = {'y':'Y','Sex':'X_Sex','Age':'X_Age'}
        rename.update({k:'X_{}'.format(k) for k in add_data.keys() if k.startswith('Clinical_')})
        rename.update({k:'X_{}'.format(k) for k in add_data.keys() if k.startswith('Diagnosis_')})
        rename.update({k:'T_{}'.format(k) for k in add_data.keys() if k.startswith('Treatment_')})
        new_df = pd.concat([new_df, add_data.drop(columns=['eid','event_dt','T2D_diagnosis_date']).rename(columns=rename)])

    new_df = new_df.sort_values(by=['task','task_train'])
    print(new_df.info())
    new_df.to_csv(f'data/processed/ukbb_dataset{dataset}.csv', index=None)
